In [6]:
import pandas as pd
#from sklearn.preprocessing import StandardScaler # [cite: 7]

# 1. Chargement et conversion propre
df = pd.read_csv('H_75_2010-2019.csv.gz', compression='gzip', sep=';', low_memory=False)
df['DATE_FULL'] = pd.to_datetime(df['AAAAMMJJHH'], format='%Y%m%d%H')

# Filtre Juillet à Décembre 2019
df_juillet = df[(df['DATE_FULL'] >= '2019-07-01') & (df['DATE_FULL'] <= '2019-12-31')].copy()
df_juillet['JOUR'] = df_juillet['DATE_FULL'].dt.normalize()

# 2. Agrégation quotidienne 
df_meteo = df_juillet.groupby('JOUR').agg({
    'T': 'mean', 
    'RR1': 'sum', 
    'U': 'mean', 
    'FF': 'mean', 
    'VV': 'mean'
}).reset_index()

# 3. Correction de la Température 
# On vérifie si T est bien en Kelvin (> 200). Sinon, on ne soustrait pas 273.15.
df_meteo['Température'] = df_meteo['T'].apply(lambda x: x - 273.15 if x > 100 else x)

# 4. Renommage des colonnes [cite: 14]
df_meteo = df_meteo.rename(columns={'RR1': 'Précipitations', 'U': 'Humidité', 'FF': 'Vent', 'VV': 'Visibilité'})
df_meteo = df_meteo.drop(columns=['T']).round(2)

print(df_meteo.head())


        JOUR  Précipitations  Humidité  Vent  Visibilité  Température
0 2019-07-01             0.0     59.35  4.24    19998.21        19.72
1 2019-07-02             0.0     55.40  4.46    19978.62        19.76
2 2019-07-03             0.0     50.94  5.73    19990.29        20.07
3 2019-07-04             0.0     46.54  4.44    19977.25        21.10
4 2019-07-05             0.0     44.88  2.50    19917.83        23.97


In [7]:
df_meteo.to_csv('meteo_juillet_decembre_2019.csv', index=False)

In [8]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   JOUR            184 non-null    datetime64[ns]
 1   Précipitations  184 non-null    float64       
 2   Humidité        184 non-null    float64       
 3   Vent            184 non-null    float64       
 4   Visibilité      184 non-null    float64       
 5   Température     184 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 8.8 KB
